**Разметка частей речи**

Евгений Борисов <esborisov@sevsu.ru>

библиотека NLTK

In [ ]:
import nltk
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')

In [8]:
sentence = 'Today morning, Arthur felt very good.'

nltk.pos_tag(  nltk.word_tokenize(sentence), lang='eng')

[('Today', 'NN'),
 ('morning', 'NN'),
 (',', ','),
 ('Arthur', 'NNP'),
 ('felt', 'VBD'),
 ('very', 'RB'),
 ('good', 'JJ'),
 ('.', '.')]

In [7]:
sentence = 'Кроме того, иногда одно и то же слово может иметь несколько разных лемм.'

nltk.pos_tag(  nltk.word_tokenize(sentence), lang='rus')

[('Кроме', 'PR'),
 ('того', 'S-PRO'),
 (',', 'NONLEX'),
 ('иногда', 'ADV'),
 ('одно', 'A-PRO=n'),
 ('и', 'CONJ'),
 ('то', 'S-PRO'),
 ('же', 'PART'),
 ('слово', 'S'),
 ('может', 'V'),
 ('иметь', 'V'),
 ('несколько', 'NUM=acc'),
 ('разных', 'A=pl'),
 ('лемм', 'S'),
 ('.', 'NONLEX')]